<a href="https://colab.research.google.com/github/Saumye13/Weather-Analysis/blob/main/Weather_Analysis_using_Parallelism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os
import time
import concurrent.futures

In [ ]:
# Execute only when do not want to use GPU

os.environ['CUDA_VISIBLE_DEVICES'] = ''

# Now TensorFlow will use only the CPU

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
import multiprocessing

In [ ]:
# Check CPU information
cpu_info = !cat /proc/cpuinfo
cpu_info = '\n'.join(cpu_info)
print("CPU Info:\n", cpu_info)

# Get the number of CPU threads
num_cpu_threads = multiprocessing.cpu_count()
print("\nNumber of CPU threads:", num_cpu_threads)


CPU Info:
 processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Check if GPU is available
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU is available')
else:
    print("GPU is NOT available")

GPU is NOT available


# Data Handling

In [ ]:
# Load the dataset
data = pd.read_csv('kanpur.csv', parse_dates=['date_time'], index_col='date_time')
features = ['maxtempC', 'mintempC', 'cloudcover', 'humidity', 'tempC', 'sunHour', 'HeatIndexC', 'precipMM', 'pressure', 'windspeedKmph']
target_variable = 'tempC'
data = data[features + [target_variable]]

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split data into input (X) and target (y) variables
X = []
y = []
sequence_length = 50

for i in range(len(data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length, :-1])
    y.append(scaled_data[i+sequence_length, -1])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X.shape

(96382, 50, 10)

In [ ]:
y.shape

(96382,)

In [ ]:
X_train.shape

(72286, 50, 10)

In [ ]:
y_train.shape

(72286,)

# Comapring Running Time on CPU and GPU

In [ ]:
# Define the LSTM model
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 50)            12200     
                                                                 
 dropout (Dropout)           (None, 50, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 50, 50)            20200     
                                                                 
 dropout_1 (Dropout)         (None, 50, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 5

## CPU

In [ ]:
# Train the model on CPU
with tf.device('/cpu:0'):
    start_time_cpu = time.time()
    history_cpu = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=2)
    end_time_cpu = time.time()

# Evaluate the model on CPU
with tf.device('/cpu:0'):
    y_pred_cpu = model.predict(X_test)
    mse_cpu = mean_squared_error(y_test, y_pred_cpu)
    print("Mean Squared Error (CPU):", mse_cpu)
    print("Time taken on CPU: {:.2f} seconds".format(end_time_cpu - start_time_cpu))


Epoch 1/10
2259/2259 - 185s - loss: 0.0034 - val_loss: 8.0700e-04 - 185s/epoch - 82ms/step
Epoch 2/10
2259/2259 - 176s - loss: 0.0012 - val_loss: 5.6315e-04 - 176s/epoch - 78ms/step
Epoch 3/10
2259/2259 - 175s - loss: 7.7305e-04 - val_loss: 4.4086e-04 - 175s/epoch - 77ms/step
Epoch 4/10
2259/2259 - 174s - loss: 6.1895e-04 - val_loss: 2.7806e-04 - 174s/epoch - 77ms/step
Epoch 5/10
2259/2259 - 175s - loss: 5.1317e-04 - val_loss: 2.5189e-04 - 175s/epoch - 78ms/step
Epoch 6/10
2259/2259 - 174s - loss: 4.7255e-04 - val_loss: 2.1822e-04 - 174s/epoch - 77ms/step
Epoch 7/10
2259/2259 - 175s - loss: 4.3020e-04 - val_loss: 1.7577e-04 - 175s/epoch - 77ms/step
Epoch 8/10
2259/2259 - 174s - loss: 4.2471e-04 - val_loss: 2.0486e-04 - 174s/epoch - 77ms/step
Epoch 9/10
2259/2259 - 175s - loss: 4.0275e-04 - val_loss: 2.5391e-04 - 175s/epoch - 78ms/step
Epoch 10/10
2259/2259 - 174s - loss: 3.9205e-04 - val_loss: 1.4936e-04 - 174s/epoch - 77ms/step
753/753 [==============================] - 19s 24ms/step


## GPU

In [ ]:
# Train the model on GPU
with tf.device('/GPU:0'):
    start_time_gpu = time.time()
    history_gpu = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=2)
    end_time_gpu = time.time()

# Evaluate the model on GPU
with tf.device('/GPU:0'):
    y_pred_gpu = model.predict(X_test)
    mse_gpu = mean_squared_error(y_test, y_pred_gpu)
    print("Mean Squared Error (GPU):", mse_gpu)
    print("Time taken on GPU: {:.2f} seconds".format(end_time_gpu - start_time_gpu))


Epoch 1/10
2260/2260 - 18s - loss: 8.7360e-04 - val_loss: 3.6659e-04 - 18s/epoch - 8ms/step
Epoch 2/10
2260/2260 - 17s - loss: 5.9986e-04 - val_loss: 3.3670e-04 - 17s/epoch - 8ms/step
Epoch 3/10
2260/2260 - 19s - loss: 5.3973e-04 - val_loss: 2.1989e-04 - 19s/epoch - 8ms/step
Epoch 4/10
2260/2260 - 17s - loss: 4.9361e-04 - val_loss: 1.7516e-04 - 17s/epoch - 8ms/step
Epoch 5/10
2260/2260 - 18s - loss: 4.6845e-04 - val_loss: 2.0474e-04 - 18s/epoch - 8ms/step
Epoch 6/10
2260/2260 - 22s - loss: 4.4861e-04 - val_loss: 1.9677e-04 - 22s/epoch - 10ms/step
Epoch 7/10
2260/2260 - 18s - loss: 4.3467e-04 - val_loss: 1.8360e-04 - 18s/epoch - 8ms/step
Epoch 8/10
2260/2260 - 18s - loss: 4.3367e-04 - val_loss: 2.0425e-04 - 18s/epoch - 8ms/step
Epoch 9/10
2260/2260 - 18s - loss: 4.2453e-04 - val_loss: 1.9720e-04 - 18s/epoch - 8ms/step
Epoch 10/10
2260/2260 - 18s - loss: 4.1256e-04 - val_loss: 2.2328e-04 - 18s/epoch - 8ms/step
754/754 [==============================] - 4s 4ms/step
Mean Squared Error (GPU

## Paralleling Training Model on Multiple GPUs


In [ ]:

# Define the distributed strategy
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Define the LSTM model within the strategy's scope
with strategy.scope():
    # Define the LSTM model
    model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=1)
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

Number of devices: 1


In [ ]:

# Define the distributed strategy
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Define the LSTM model within the strategy's scope
with strategy.scope():
    # Define the LSTM model
    model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=1)
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    start_time_m_gpu = time.time()
    history_m_gpu = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    end_time_m_gpu = time.time()

    # Evaluate the model
    y_pred_m_gpu = model.predict(X_test)
    mse_m_gpu = mean_squared_error(y_test, y_pred_m_gpu)
    print("Mean Squared Error:", mse_m_gpu)
    print("Time taken on Multiple GPU: {:.2f} seconds".format(end_time_m_gpu - start_time_m_gpu))


Number of devices: 1
Epoch 1/10
2260/2260 [==============================] - 65s 27ms/step - loss: 0.0042 - val_loss: 8.9936e-04
Epoch 2/10
2260/2260 [==============================] - 23s 10ms/step - loss: 0.0012 - val_loss: 3.5314e-04
Epoch 3/10
2260/2260 [==============================] - 26s 11ms/step - loss: 6.6823e-04 - val_loss: 2.3051e-04
Epoch 4/10
2260/2260 [==============================] - 25s 11ms/step - loss: 5.6425e-04 - val_loss: 2.4551e-04
Epoch 5/10
2260/2260 [==============================] - 25s 11ms/step - loss: 5.1157e-04 - val_loss: 2.2027e-04
Epoch 6/10
2260/2260 [==============================] - 23s 10ms/step - loss: 4.7403e-04 - val_loss: 1.9500e-04
Epoch 7/10
2260/2260 [==============================] - 23s 10ms/step - loss: 4.5973e-04 - val_loss: 2.1746e-04
Epoch 8/10
2260/2260 [==============================] - 25s 11ms/step - loss: 4.4274e-04 - val_loss: 2.3283e-04
Epoch 9/10
2260/2260 [==============================] - 23s 10ms/step - loss: 4.3618e-04 - 

In [ ]:
# Train the model
start_time_m_gpu = time.time()
history_m_gpu = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
end_time_m_gpu = time.time()

# Evaluate the model
y_pred_m_gpu = model.predict(X_test)
mse_m_gpu = mean_squared_error(y_test, y_pred_m_gpu)
print("Mean Squared Error:", mse_m_gpu)
print("Time taken on Multiple GPU: {:.2f} seconds".format(end_time_m_gpu - start_time_m_gpu))

Epoch 1/10
2260/2260 [==============================] - 29s 11ms/step - loss: 0.0040 - val_loss: 0.0014
Epoch 2/10
2260/2260 [==============================] - 23s 10ms/step - loss: 0.0012 - val_loss: 3.5822e-04
Epoch 3/10
2260/2260 [==============================] - 23s 10ms/step - loss: 6.8586e-04 - val_loss: 3.2943e-04
Epoch 4/10
2260/2260 [==============================] - 23s 10ms/step - loss: 5.6491e-04 - val_loss: 2.2612e-04
Epoch 5/10
2260/2260 [==============================] - 23s 10ms/step - loss: 5.1611e-04 - val_loss: 3.0820e-04
Epoch 6/10
2260/2260 [==============================] - 23s 10ms/step - loss: 4.8201e-04 - val_loss: 1.8628e-04
Epoch 7/10
2260/2260 [==============================] - 25s 11ms/step - loss: 4.6078e-04 - val_loss: 2.1599e-04
Epoch 8/10
2260/2260 [==============================] - 23s 10ms/step - loss: 4.4884e-04 - val_loss: 1.9496e-04
Epoch 9/10
2260/2260 [==============================] - 23s 10ms/step - loss: 4.4070e-04 - val_loss: 3.9716e-04
Epoc

# Multi Threading (Sequence Length = 10)

In [ ]:
# Function to define and train an LSTM model
def train_lstm_model(X_train, y_train, X_test, y_test, units, layers, dropout_rate):
    # Define the LSTM model with dropout layers
    model = Sequential()
    for _ in range(layers - 1):
        model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    start_time = time.time()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    end_time = time.time()

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse, end_time - start_time

### Sequence Length = 10

In [ ]:

# Load the dataset
data = pd.read_csv('kanpur.csv', parse_dates=['date_time'], index_col='date_time')
features = ['maxtempC', 'mintempC', 'cloudcover', 'humidity', 'tempC', 'sunHour', 'HeatIndexC', 'precipMM', 'pressure', 'windspeedKmph']
target_variable = 'tempC'
data = data[features + [target_variable]]

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split data into input (X) and target (y) variables
X = []
y = []
sequence_length = 10

for i in range(len(data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length, :-1])
    y.append(scaled_data[i+sequence_length, -1])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define model configurations
model_configurations = [
    {'units': 50, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 50, 'layers': 2, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 2, 'dropout_rate': 0.2}
]

## Single Thread (Sequence Length = 10)

### Without main Function

In [ ]:
# Set up a concurrent futures executor with 1 CPU threads
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    # Submit tasks for each model configuration
    futures = {executor.submit(train_lstm_model, X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate']): config for config in model_configurations}

    # Collect results as they become available
    for future in concurrent.futures.as_completed(futures):
        config = futures[future]
        mse, training_time = future.result()
        print("Model - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(config['units'], config['layers'], config['dropout_rate'], mse, training_time))


603/603 [==============================] - 4s 5ms/step
Model - Units: 50, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000171, Training Time: 264.32 seconds
603/603 [==============================] - 4s 6ms/step
Model - Units: 50, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000165, Training Time: 385.51 seconds
603/603 [==============================] - 4s 5ms/step
Model - Units: 100, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000159, Training Time: 324.47 seconds
603/603 [==============================] - 6s 9ms/step
Model - Units: 100, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000178, Training Time: 626.44 seconds


### With main function (Windows Interative Interpreter)

In [ ]:
if __name__ == '__main__':
    # Set up a concurrent futures executor with 1 CPU threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        # Submit tasks for each model configuration
        futures = {executor.submit(train_lstm_model, X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate']): config for config in model_configurations}

        # Collect results as they become available
        for future in concurrent.futures.as_completed(futures):
            config = futures[future]
            mse, training_time = future.result()
            print("Model - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(config['units'], config['layers'], config['dropout_rate'], mse, training_time))


603/603 [==============================] - 3s 5ms/step
Model - Units: 50, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000171, Training Time: 265.65 seconds
603/603 [==============================] - 4s 6ms/step
Model - Units: 50, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000203, Training Time: 377.64 seconds
603/603 [==============================] - 4s 6ms/step
Model - Units: 100, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000154, Training Time: 293.43 seconds
603/603 [==============================] - 6s 8ms/step
Model - Units: 100, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000160, Training Time: 565.44 seconds


## Multiple Threads (Sequence Length = 10)

### Without main Function

In [ ]:
# Set up a concurrent futures executor with 2 CPU threads
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    # Submit tasks for each model configuration
    futures = {executor.submit(train_lstm_model, X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate']): config for config in model_configurations}

    # Collect results as they become available
    for future in concurrent.futures.as_completed(futures):
        config = futures[future]
        mse, training_time = future.result()
        print("Model - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(config['units'], config['layers'], config['dropout_rate'], mse, training_time))


603/603 [==============================] - 5s 6ms/step
Model - Units: 50, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000221, Training Time: 333.40 seconds
603/603 [==============================] - 6s 8ms/step
Model - Units: 50, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000218, Training Time: 588.55 seconds
603/603 [==============================] - 6s 8ms/step
Model - Units: 100, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000172, Training Time: 448.21 seconds
603/603 [==============================] - 7s 10ms/step
Model - Units: 100, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000182, Training Time: 630.41 seconds


### With main function (Windows Interative Interpreter)

In [ ]:

if __name__ == '__main__':
    # Load the dataset
    data = pd.read_csv('kanpur.csv', parse_dates=['date_time'], index_col='date_time')
    features = ['maxtempC', 'mintempC', 'cloudcover', 'humidity', 'tempC', 'sunHour', 'HeatIndexC', 'precipMM', 'pressure', 'windspeedKmph']
    target_variable = 'tempC'
    data = data[features + [target_variable]]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Split data into input (X) and target (y) variables
    X = []
    y = []
    sequence_length = 10

    for i in range(len(data) - sequence_length):
        X.append(scaled_data[i:i+sequence_length, :-1])
        y.append(scaled_data[i+sequence_length, -1])

    X = np.array(X)
    y = np.array(y)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define model configurations
    model_configurations = [
        {'units': 50, 'layers': 1, 'dropout_rate': 0.2},
        {'units': 50, 'layers': 2, 'dropout_rate': 0.2},
        {'units': 100, 'layers': 1, 'dropout_rate': 0.2},
        {'units': 100, 'layers': 2, 'dropout_rate': 0.2}
    ]

    # Set up a concurrent futures executor with 2 CPU threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        # Submit tasks for each model configuration
        futures = {executor.submit(train_lstm_model, X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate']): config for config in model_configurations}

        # Collect results as they become available
        for future in concurrent.futures.as_completed(futures):
            config = futures[future]
            mse, training_time = future.result()
            print("Model - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(config['units'], config['layers'], config['dropout_rate'], mse, training_time))


603/603 [==============================] - 14s 22ms/step
Model - Units: 50, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000235, Training Time: 873.71 seconds
603/603 [==============================] - 15s 20ms/step
Model - Units: 50, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000165, Training Time: 1135.99 seconds
603/603 [==============================] - 12s 17ms/step
Model - Units: 100, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000159, Training Time: 861.72 seconds
603/603 [==============================] - 7s 10ms/step
Model - Units: 100, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000167, Training Time: 1054.20 seconds


# Multi Threading (Sequence Length = 50)

In [ ]:
# Function to define and train an LSTM model
def train_lstm_model(X_train, y_train, X_test, y_test, units, layers, dropout_rate):
    # Define the LSTM model with dropout layers
    model = Sequential()
    for _ in range(layers - 1):
        model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    start_time = time.time()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    end_time = time.time()

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse, end_time - start_time

### Sequence Length = 50

In [ ]:

# Load the dataset
data = pd.read_csv('kanpur.csv', parse_dates=['date_time'], index_col='date_time')
features = ['maxtempC', 'mintempC', 'cloudcover', 'humidity', 'tempC', 'sunHour', 'HeatIndexC', 'precipMM', 'pressure', 'windspeedKmph']
target_variable = 'tempC'
data = data[features + [target_variable]]

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split data into input (X) and target (y) variables
X = []
y = []
sequence_length = 50

for i in range(len(data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length, :-1])
    y.append(scaled_data[i+sequence_length, -1])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define model configurations
model_configurations = [
    {'units': 50, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 50, 'layers': 2, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 2, 'dropout_rate': 0.2}
]

## Single Thread (Sequence Length = 50)

### Without main Function

In [ ]:
# Set up a concurrent futures executor with 1 CPU threads
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    # Submit tasks for each model configuration
    futures = {executor.submit(train_lstm_model, X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate']): config for config in model_configurations}

    # Collect results as they become available
    for future in concurrent.futures.as_completed(futures):
        config = futures[future]
        mse, training_time = future.result()
        print("Model - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(config['units'], config['layers'], config['dropout_rate'], mse, training_time))


603/603 [==============================] - 5s 8ms/step
Model - Units: 50, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000206, Training Time: 504.81 seconds
603/603 [==============================] - 9s 14ms/step
Model - Units: 50, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000157, Training Time: 1045.75 seconds
603/603 [==============================] - 8s 12ms/step
Model - Units: 100, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000129, Training Time: 733.33 seconds
603/603 [==============================] - 16s 25ms/step
Model - Units: 100, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000250, Training Time: 1765.49 seconds


### With main function (Windows Interative Interpreter)

In [ ]:
if __name__ == '__main__':
    # Set up a concurrent futures executor with 1 CPU threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        # Submit tasks for each model configuration
        futures = {executor.submit(train_lstm_model, X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate']): config for config in model_configurations}

        # Collect results as they become available
        for future in concurrent.futures.as_completed(futures):
            config = futures[future]
            mse, training_time = future.result()
            print("Model - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(config['units'], config['layers'], config['dropout_rate'], mse, training_time))


603/603 [==============================] - 6s 9ms/step
Model - Units: 50, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000141, Training Time: 625.33 seconds
603/603 [==============================] - 12s 18ms/step
Model - Units: 50, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000192, Training Time: 1346.24 seconds
603/603 [==============================] - 10s 16ms/step
Model - Units: 100, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000136, Training Time: 924.94 seconds
603/603 [==============================] - 22s 35ms/step
Model - Units: 100, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000165, Training Time: 2186.51 seconds


## Multiple Threads (Sequence Length = 50)

### Without main Function

In [ ]:
# Set up a concurrent futures executor with 2 CPU threads
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    # Submit tasks for each model configuration
    futures = {executor.submit(train_lstm_model, X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate']): config for config in model_configurations}

    # Collect results as they become available
    for future in concurrent.futures.as_completed(futures):
        config = futures[future]
        mse, training_time = future.result()
        print("Model - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(config['units'], config['layers'], config['dropout_rate'], mse, training_time))


603/603 [==============================] - 10s 15ms/step
Model - Units: 50, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000189, Training Time: 1110.11 seconds
603/603 [==============================] - 19s 26ms/step
Model - Units: 50, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000177, Training Time: 2253.31 seconds
603/603 [==============================] - 14s 22ms/step
Model - Units: 100, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000140, Training Time: 1527.48 seconds
603/603 [==============================] - 21s 33ms/step
Model - Units: 100, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000142, Training Time: 2334.52 seconds


### With main function (Windows Interative Interpreter)

In [ ]:

if __name__ == '__main__':
    # Load the dataset
    data = pd.read_csv('kanpur.csv', parse_dates=['date_time'], index_col='date_time')
    features = ['maxtempC', 'mintempC', 'cloudcover', 'humidity', 'tempC', 'sunHour', 'HeatIndexC', 'precipMM', 'pressure', 'windspeedKmph']
    target_variable = 'tempC'
    data = data[features + [target_variable]]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Split data into input (X) and target (y) variables
    X = []
    y = []
    sequence_length = 50

    for i in range(len(data) - sequence_length):
        X.append(scaled_data[i:i+sequence_length, :-1])
        y.append(scaled_data[i+sequence_length, -1])

    X = np.array(X)
    y = np.array(y)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define model configurations
    model_configurations = [
        {'units': 50, 'layers': 1, 'dropout_rate': 0.2},
        {'units': 50, 'layers': 2, 'dropout_rate': 0.2},
        {'units': 100, 'layers': 1, 'dropout_rate': 0.2},
        {'units': 100, 'layers': 2, 'dropout_rate': 0.2}
    ]

    # Set up a concurrent futures executor with 2 CPU threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        # Submit tasks for each model configuration
        futures = {executor.submit(train_lstm_model, X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate']): config for config in model_configurations}

        # Collect results as they become available
        for future in concurrent.futures.as_completed(futures):
            config = futures[future]
            mse, training_time = future.result()
            print("Model - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(config['units'], config['layers'], config['dropout_rate'], mse, training_time))


603/603 [==============================] - 7s 10ms/step
Model - Units: 50, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000134, Training Time: 830.10 seconds
603/603 [==============================] - 15s 22ms/step
Model - Units: 50, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000155, Training Time: 1830.94 seconds
603/603 [==============================] - 11s 17ms/step
Model - Units: 100, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000208, Training Time: 1219.53 seconds
603/603 [==============================] - 16s 25ms/step
Model - Units: 100, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000156, Training Time: 1888.93 seconds


# Single Thread

In [ ]:
# Function to define and train an LSTM model
def train_lstm_model(X_train, y_train, X_test, y_test, units, layers, dropout_rate):
    # Define the LSTM model with dropout layers
    model = Sequential()
    for _ in range(layers - 1):
        model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    start_time = time.time()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    end_time = time.time()

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse, end_time - start_time

In [ ]:


# Load the dataset
data = pd.read_csv('kanpur.csv', parse_dates=['date_time'], index_col='date_time')
features = ['maxtempC', 'mintempC', 'cloudcover', 'humidity', 'tempC', 'sunHour', 'HeatIndexC', 'precipMM', 'pressure', 'windspeedKmph']
target_variable = 'tempC'
data = data[features + [target_variable]]

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split data into input (X) and target (y) variables
X = []
y = []
sequence_length = 10

for i in range(len(data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length, :-1])
    y.append(scaled_data[i+sequence_length, -1])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define model configurations
model_configurations = [
    {'units': 50, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 50, 'layers': 2, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 2, 'dropout_rate': 0.2}
]

In [ ]:
# Run each model configuration sequentially on a single thread
for config in model_configurations:
    mse, training_time = train_lstm_model(X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate'])
    print("Model - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(config['units'], config['layers'], config['dropout_rate'], mse, training_time))


603/603 [==============================] - 2s 3ms/step
Model - Units: 50, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000187, Training Time: 170.02 seconds
603/603 [==============================] - 3s 4ms/step
Model - Units: 50, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000289, Training Time: 325.24 seconds
603/603 [==============================] - 3s 4ms/step
Model - Units: 100, Layers: 1, Dropout Rate: 0.2, Mean Squared Error: 0.000177, Training Time: 264.10 seconds
603/603 [==============================] - 5s 7ms/step
Model - Units: 100, Layers: 2, Dropout Rate: 0.2, Mean Squared Error: 0.000206, Training Time: 441.89 seconds


# Training Mutiple Models as Different Processes in Parallel

### Single CPU Process (Deadlock)

In [ ]:
# Function to define and train an LSTM model
def train_lstm_model(X_train, y_train, X_test, y_test, units, layers, dropout_rate):
    # Define the LSTM model with dropout layers
    model = Sequential()
    for _ in range(layers - 1):
        model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    model.summary()

    # Train the model
    start_time = time.time()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    end_time = time.time()

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse, end_time - start_time


In [ ]:
# Define model configurations
model_configurations = [
    {'units': 50, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 50, 'layers': 2, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 2, 'dropout_rate': 0.2}
]


### Without Specifying Device as CPU

In [ ]:
# Train and evaluate each model sequentially
for i, config in enumerate(model_configurations):
    print(f"Training Model {i+1}...")
    mse, training_time = train_lstm_model(X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate'])
    print("Model {} - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds\n".format(i+1, config['units'], config['layers'], config['dropout_rate'], mse, training_time))


### Specifying Device as CPU

In [ ]:
with tf.device('/cpu:0'):
    # Train and evaluate each model sequentially
    for i, config in enumerate(model_configurations):
        print(f"Training Model {i+1}...")
        mse, training_time = train_lstm_model(X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate'])
        print("Model {} - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds\n".format(i+1, config['units'], config['layers'], config['dropout_rate'], mse, training_time))


### Multiple CPU Processes (Deadlock)

In [ ]:
# Function to define and train an LSTM model
def train_lstm_model(X_train, y_train, X_test, y_test, units, layers, dropout_rate, result_queue):
    # Define the LSTM model with dropout layers
    model = Sequential()
    for _ in range(layers - 1):
        model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    start_time = time.time()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    end_time = time.time()

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Put results in queue
    result_queue.put((mse, end_time - start_time))


In [ ]:

# Define model configurations
model_configurations = [
    {'units': 50, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 50, 'layers': 2, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 2, 'dropout_rate': 0.2}
]


### Without Specifying Device as CPU

In [ ]:

# Set up a multiprocessing queue for collecting results
result_queue = multiprocessing.Queue()

# Create and start a process for each model
processes = []
for config in model_configurations:
    process = multiprocessing.Process(target=train_lstm_model, args=(X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate'], result_queue))
    processes.append(process)
    process.start()

# Wait for all processes to finish
for process in processes:
    process.join()

# Collect and print results
for i, config in enumerate(model_configurations):
    mse, training_time = result_queue.get()
    print("Model {} - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(i+1, config['units'], config['layers'], config['dropout_rate'], mse, training_time))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


KeyboardInterrupt: 

### Specifying Device as CPU

In [ ]:
with tf.device('/cpu:0'):
    # Set up a multiprocessing queue for collecting results
    result_queue = multiprocessing.Queue()

    # Create and start a process for each model
    processes = []
    for config in model_configurations:
        process = multiprocessing.Process(target=train_lstm_model, args=(X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate'], result_queue))
        processes.append(process)
        process.start()

    # Wait for all processes to finish
    for process in processes:
        process.join()

    # Collect and print results
    for i, config in enumerate(model_configurations):
        mse, training_time = result_queue.get()
        print("Model {} - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(i+1, config['units'], config['layers'], config['dropout_rate'], mse, training_time))


KeyboardInterrupt: 

# Training Multiple Models on Multiple Threads in Parallel

In [ ]:
import threading
import queue

### Multiple CPU Threads (Does Not Work)

In [ ]:
# Function to define and train an LSTM model
def train_lstm_model(X_train, y_train, X_test, y_test, units, layers, dropout_rate, result_queue):
    # Define the LSTM model with dropout layers
    model = Sequential()
    for _ in range(layers - 1):
        model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    start_time = time.time()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    end_time = time.time()

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Put results in queue
    result_queue.put((mse, end_time - start_time))


In [ ]:

# Define model configurations
model_configurations = [
    {'units': 50, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 50, 'layers': 2, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 1, 'dropout_rate': 0.2},
    {'units': 100, 'layers': 2, 'dropout_rate': 0.2}
]


### Without Specifying Device as CPU

In [ ]:

# Set up a threading queue for collecting results
result_queue = queue.Queue()

# Create and start a thread for each model
threads = []
for config in model_configurations:
    thread = threading.Thread(target=train_lstm_model, args=(X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate'], result_queue))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

# Collect and print results
for i, config in enumerate(model_configurations):
    mse, training_time = result_queue.get()
    print("Model {} - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(i+1, config['units'], config['layers'], config['dropout_rate'], mse, training_time))


KeyboardInterrupt: 

### Specifying Device as CPU

In [ ]:
with tf.device('/cpu:0'):
    # Set up a threading queue for collecting results
    result_queue = queue.Queue()

    # Create and start a thread for each model
    threads = []
    for config in model_configurations:
        thread = threading.Thread(target=train_lstm_model, args=(X_train, y_train, X_test, y_test, config['units'], config['layers'], config['dropout_rate'], result_queue))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    # Collect and print results
    for i, config in enumerate(model_configurations):
        mse, training_time = result_queue.get()
        print("Model {} - Units: {}, Layers: {}, Dropout Rate: {}, Mean Squared Error: {:.6f}, Training Time: {:.2f} seconds".format(i+1, config['units'], config['layers'], config['dropout_rate'], mse, training_time))
